Get Target Id with characteristics of icd10, tag, attribute, value


In [1]:
from ClinicalDocumentUtils import Database
import numpy as np
import pandas as pd
from utils.DB import DB

In [2]:
d = Database()

Connect To Database


encryption key: ········


Connected: {'user': 'postgres', 'channel_binding': 'prefer', 'dbname': 'clinical', 'host': 'clinical_db', 'port': '5432', 'options': '-c search_path=clinical_document,public', 'sslmode': 'require', 'sslcompression': '0', 'sslcertmode': 'allow', 'sslsni': '1', 'ssl_min_protocol_version': 'TLSv1.2', 'gssencmode': 'prefer', 'krbsrvname': 'postgres', 'gssdelegation': '0', 'target_session_attrs': 'any', 'load_balance_hosts': 'disable'}


In [3]:
d.fetch_projects()

,name,description,uuid
0,test,test,00e50190-f573-4750-8946-70b70f2ca70c


In [4]:
d.set_project_uuid('00e50190-f573-4750-8946-70b70f2ca70c')

project_uuid: 00e50190-f573-4750-8946-70b70f2ca70c


In [5]:
filters = [
{'icd10':None, 'tag': 'diagnosis', 'attribute':'diagnosis', 'value':['c44.92 - Squamous cell carcinoma of skin, unspecified','c44.91 - Basal cell carcinoma of skin, unspecified']},
]
d.get_mrns_where_filters(filters)

['s222222', 's111111']

In [6]:
db = DB()

Connect To Database


encryption key: ········


Connected: {'user': 'postgres', 'channel_binding': 'prefer', 'dbname': 'clinical', 'host': 'clinical_db', 'port': '5432', 'options': '-c search_path=clinical_document,public', 'sslmode': 'require', 'sslcompression': '0', 'sslcertmode': 'allow', 'sslsni': '1', 'ssl_min_protocol_version': 'TLSv1.2', 'gssencmode': 'prefer', 'krbsrvname': 'postgres', 'gssdelegation': '0', 'target_session_attrs': 'any', 'load_balance_hosts': 'disable'}


In [7]:
# build OR sql query
def build_logical_OR_LIKE_sql_query(column, values):
  phase1 = f'\' OR {column} LIKE \''.join(values)
  sql = f"{column} LIKE \'{phase1}\'"
  return sql

In [46]:
def does_target_filter_exist(mrn, target_id, filters):
    # Build SQL Logic
    for idx, filter in enumerate(filters):
        #print(filter)
        logic = ''
        
        # New Search Term
        #if idx != 0: logic = logic + ' OR'
        
        # If icd10 exists
        if 'icd10' in filter.keys() and filter['icd10'] != None: 
            #print(f"icd10:{filter['icd10']}")
            logic = logic + f" (icd10 LIKE '{filter['icd10']}' "
        else:
            logic = logic + f" (icd10 IS NULL "
            
        
        # If icd10 exists
        if 'tag' in filter.keys(): 
            #print(f"tag:{filter['tag']}")
            logic = logic + f"AND tag LIKE '{filter['tag']}' "
    
        # If attribute exists
        if 'attribute' in filter.keys(): 
            #print(f"attribute:{filter['attribute']}")
            logic = logic + f"AND obj->>'name' LIKE '{filter['attribute']}' "
    
        # If attribute values exists
        if 'value' in filter.keys(): 
            #print(f"attribute values:{filter['value']}")
            logic = logic +'AND ('+ build_logical_OR_LIKE_sql_query(column="obj->>'value'", values=filter["value"]) +')'
        logic = logic + ')'

        print(logic)
        # Does mrn-target meet criteria
        cursor = db.connection.cursor()
        sql = f"SELECT target.attribute \
                FROM \
                    (SELECT q.icd10, q.tag, q.attribute \
                    FROM clinical_document.q_document as q, jsonb_array_elements(q.attribute) as attr \
                    WHERE q.mrn='{mrn}' AND attr->>'name'='target-id' AND attr->>'value'='{target_id}') target, \
                    jsonb_array_elements(target.attribute) as obj \
                WHERE {logic} "
        #print(sql)
        cursor.execute(sql)
        count = cursor.fetchone()
        cursor.close()
        print
        if count == None: return False
    return True

def get_mrn_targets_where_filter(filters):
    project_uuid = '00e50190-f573-4750-8946-70b70f2ca70c'
    cursor = db.connection.cursor()
    
    # Get All MRNs & Targets
    sql = f"SELECT DISTINCT q.mrn as mrn, attr->>'value' as target \
            FROM clinical_document.q_document as q, jsonb_array_elements(q.attribute) as attr \
            WHERE attr->>'name'='target-id' AND q.project_uuid='{project_uuid}'"
    cursor.execute(sql)
    mrn_targets = np.asarray(cursor.fetchall())
    #print(mrn_targets)
    valids = np.full(shape=len(mrn_targets), fill_value=False)

    cursor.close()

    # Loop through MRNs
    filter_exists = np.full(len(mrn_targets), fill_value=False, dtype=bool)
    for idx, (mrn, target_id) in enumerate(mrn_targets):
        if does_target_filter_exist(mrn, target_id, filters):
           filter_exists[idx] = True 
    
    return mrn_targets[filter_exists]
    
filters=[{'icd10':'c44.91', 'tag': 'histology', 'attribute': 'histology', 'value': ['nodular']},
        {'icd10':'c44.91'}]

get_mrn_targets_where_filter(filters)

 (icd10 LIKE 'c44.91' AND tag LIKE 'histology' AND obj->>'name' LIKE 'histology' AND (obj->>'value' LIKE 'nodular'))
 (icd10 LIKE 'c44.91' )
 (icd10 LIKE 'c44.91' AND tag LIKE 'histology' AND obj->>'name' LIKE 'histology' AND (obj->>'value' LIKE 'nodular'))
 (icd10 LIKE 'c44.91' )
 (icd10 LIKE 'c44.91' AND tag LIKE 'histology' AND obj->>'name' LIKE 'histology' AND (obj->>'value' LIKE 'nodular'))
 (icd10 LIKE 'c44.91' AND tag LIKE 'histology' AND obj->>'name' LIKE 'histology' AND (obj->>'value' LIKE 'nodular'))


array([['s222222', 'right-cheek'],
       ['s222222', 'left-temple']], dtype='<U17')

In [14]:

sql = ''
# Build SQL Logic
# Loop through tags
for idx, filter in enumerate(filter):
    #print(filter)
    
    # New Search Term
    if idx != 0: sql = sql + ' OR'
    
    # If icd10 exists
    if 'icd10' in filter.keys() and filter['icd10'] != None: 
        #print(f"icd10:{filter['icd10']}")
        sql = sql + f" (icd10 LIKE '{filter['icd10']}' "
    else:
        sql = sql + f" (icd10 IS NULL "
        
    
    # If icd10 exists
    if 'tag' in filter.keys(): 
        #print(f"tag:{filter['tag']}")
        sql = sql + f"AND tag LIKE '{filter['tag']}' "

    # If attribute exists
    if 'attribute' in filter.keys(): 
        #print(f"attribute:{filter['attribute']}")
        sql = sql + f"AND obj->>'name' LIKE '{filter['attribute']}' "

    # If attribute values exists
    if 'value' in filter.keys(): 
        #print(f"attribute values:{filter['value']}")
        sql = sql +'AND ('+ build_logical_OR_LIKE_sql_query(column="obj->>'value'", values=filter["value"]) +')'

    sql = sql + ')'
# Built SQL Search Logic
print(sql)

 (icd10 LIKE 'c44.91' AND tag LIKE 'histology' AND obj->>'name' LIKE 'histology' AND (obj->>'value' LIKE 'nodular'))
